In [36]:
import cv2
import pytesseract

# Ossen funker dette a?
- Ta screenshot av registernavnene og offsettene
- Legg dem i samme mappe som notebooken og gi bildene navnene reg.png og off.png
- Kjør alle cellene
- Len deg tilbake popp en shni og chill for du har akkurat spart deg selv for 6 min med marging

In [37]:
def finn_offset(off):
    byte_per_word = 4
    return off/byte_per_word
finn_offset((0x554 - 0x524))

12.0

In [38]:
def set_up_struct(ofsets):
    dec_list = []
    val_list = []
    reserv = 0
    for i in range(len(ofsets)):
        if i == 0:
            first = 0x0
        else:
            first = int(ofsets[i - 1], 16) + 0x4


        second = int(ofsets[i],16)
        ofset_val = finn_offset(second - first)


        if ofset_val != 0:
            if ofset_val == 1:
                val_list.append(f'reserv{reserv}')
                val_list.append('normal_reg')
            else:
                val_list.append(f'reserv{reserv}[{int(ofset_val)}]')
                val_list.append('normal_reg')
            reserv = reserv + 1
        else:
            val_list.append(f'normal_reg')

        dec_list.append(ofset_val)

    return  val_list

In [39]:

def merg(regs, struct):
    n = 0
    for i, val in enumerate(struct):
        if 'reserv' not in val:
            struct[i] = regs[n]
            n = n + 1
        struct[i] = 'volatile uint32_t ' + struct[i] + ";"
        print(struct[i])
    return struct

In [40]:
def clean_list(list):
    for i, val in enumerate(list):
        if val == '':
            del(list[i])
        if 'Ox' in val:
            list[i] = val.replace('Ox','0x')
        if 'Register' in val:
            del(list[i])
    return list

In [41]:

reg_string = pytesseract.image_to_string(cv2.imread('reg.png')).split('\n')
ofsets_string = pytesseract.image_to_string(cv2.imread('off.png')).split('\n')
print(reg_string)
print(ofsets_string)

['OUT', 'OUTSET', 'OUTCLR', '', 'IN', '', 'DIR', 'DIRSET', 'DIRCLR', 'PIN_CNF[O]', 'PIN_CNF[1]', '']
['0x504', '0x508', '0x50C', '0x510', '0x514', '0x518', '0x51C', '0x700', '0x704', '']


In [42]:

reg_list = clean_list(reg_string)
ofset_list = clean_list(ofsets_string)

basic_struct = set_up_struct(ofset_list)
print(reg_list)
print(ofset_list)

['OUT', 'OUTSET', 'OUTCLR', 'IN', 'DIR', 'DIRSET', 'DIRCLR', 'PIN_CNF[O]', 'PIN_CNF[1]']
['0x504', '0x508', '0x50C', '0x510', '0x514', '0x518', '0x51C', '0x700', '0x704']


In [43]:

STRUCT = merg(reg_list, basic_struct)

volatile uint32_t reserv0[321];
volatile uint32_t OUT;
volatile uint32_t OUTSET;
volatile uint32_t OUTCLR;
volatile uint32_t IN;
volatile uint32_t DIR;
volatile uint32_t DIRSET;
volatile uint32_t DIRCLR;
volatile uint32_t reserv1[120];
volatile uint32_t PIN_CNF[O];
volatile uint32_t PIN_CNF[1];
